In [ ]:
#pandas
import pandas as pd
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
#sklearn
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import fbeta_score,recall_score, f1_score
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import brier_score_loss
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
#imblearn
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import (
                                    RandomOverSampler, 
                                    SMOTE, 
                                    ADASYN,
                                    BorderlineSMOTE
                                   )
from imblearn.under_sampling import (
                                    RandomUnderSampler,
                                    OneSidedSelection,
                                    NeighbourhoodCleaningRule,
                                    TomekLinks
                                   )
from imblearn.combine import (SMOTEENN, SMOTETomek)
#others
import numpy as np
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import categorical_embedder as cem
import collections
import category_encoders as ce
from matplotlib import pyplot as plt
import category_encoders as ce
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
import math


#instantiate scaler
scaler = MinMaxScaler()

In [ ]:
def mean_mode_impute(df):
    impute_mean = SimpleImputer(strategy="mean")
    impute_mc = SimpleImputer(strategy="most_frequent")
    column_list = df.columns[df.isna().any()].tolist()
    if column_list == []:
#        print('No columns with missing values found')
        return df
    for column in column_list:
         if is_numeric_dtype(df[column]):
            impute_mean = impute_mean.fit(df[[column]])
            df[column] = impute_mean.transform(df[[column]]).ravel()
         elif is_string_dtype(df[column]):
            impute_mc = impute_mc.fit(df[[column]])
            df[column] = impute_mc.transform(df[[column]]).ravel()
         else:
            print('error')  
    return df 
            
def woe_encode(x,y):
    column_list = x.columns.tolist()
    for column in column_list:
        if is_string_dtype(x[column]):
            encoder = ce.WOEEncoder(cols=[column])
            encoder.fit(x,y)
            x = encoder.transform(x)
    X_woe['BAD'] = y_woe
    ld_woe = X_woe
    return ld_woe

def one_hot_encode(df):
    column_list = df.columns.tolist()
    for column in column_list:
        if is_string_dtype(df[column]):
            one_hot_column = pd.get_dummies(df[column],prefix=column,drop_first=True)
            df = df.drop(column,axis = 1)
            df = df.join(one_hot_column)
    return df

In [ ]:
def classify(X_train,y_train,X_test,y_test,resampler,parameterR, transformer, parameterT,encoder,classifier):
    
    res = []
    model_name = type(classifier).__name__
    model = classifier.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)
    #print(y_proba)
    y_proba = y_proba[:, 1]
    y_test_arr = np.array(y_test)
    rec = str(round(recall_score(y_test_arr, y_pred),4))
    f1 = str(round(metrics.f1_score(y_test, y_pred),4))
    auc = str(round(metrics.roc_auc_score(y_test, y_pred),4))
    bs = str(round(metrics.brier_score_loss(y_test, y_proba),4))
    print(dataset_name,encoder, resampler,parameterR,transformer,parameterT, model_name,rec,f1,auc,bs )
    res.append(( dataset_name,encoder, resampler,parameterR,transformer,parameterT, model_name,rec,f1,auc,bs ))
    return res


def split(X,y):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify = y)
        return X_train, X_test, y_train, y_test



def classify_master(X_train, X_test, y_train, y_test,encoder_name,classifier):   
    preprocess_technique_stat = pd.DataFrame()
    
    stat_encoded = classify(X_train,y_train,X_test,y_test,'NA','NA', 'NA', 'NA',encoder_name,classifier)
    preprocess_technique_stat = preprocess_technique_stat.append(stat_encoded)
    
    stat_resample = resample_classify(X_train,X_test, y_train,y_test, ratios,encoder_name,classifier)
    preprocess_technique_stat = preprocess_technique_stat.append(stat_resample)
    
    stat_reduce = reduce_classify(X_train,y_train,X_test,y_test,'NA', 'NA',encoder_name,classifier)
    preprocess_technique_stat = preprocess_technique_stat.append(stat_reduce)
    
    stat_combined = process_classify(X_train,X_test, y_train,y_test, ratios,encoder_name,classifier)
    preprocess_technique_stat = preprocess_technique_stat.append(stat_combined)

    return preprocess_technique_stat

In [ ]:
def process_classify(X_train,X_test, y_train,y_test, ratios,encoder,classifier):
    results = pd.DataFrame()
    k_neighbors = [5,10,15]

    
    resampler_instance = OneSidedSelection()
    pipeline = Pipeline([('resampler', resampler_instance),  ('clf', classifier)])
    params = {'resampler__sampling_strategy' : ['not minority','not majority','all'],
              'resampler__n_neighbors':[1,3,6],
             'resampler__n_seeds_S':[10,100,200]}
    grid = GridSearchCV(pipeline, params,cv=5,scoring='roc_auc')
    grid.fit(X_train, y_train)
    pRatio = grid.best_params_['resampler__sampling_strategy']
    pNeighbors = grid.best_params_['resampler__n_neighbors']
    pSeeds = grid.best_params_['resampler__n_seeds_S']
    resampler_instance = OneSidedSelection(sampling_strategy=pRatio,n_neighbors=pNeighbors,n_seeds_S=pSeeds)
    X_resampled, y_resampled = resampler_instance.fit_sample(X_train, y_train)
    stat = reduce_classify(X_resampled,y_resampled,X_test,y_test,'OSS',grid.best_params_ ,encoder,classifier)
    results = results.append(( stat ))
    
    
    ROS_instance = RandomOverSampler()
    pipeline = Pipeline([('resampler', ROS_instance), ('clf', classifier)])
    params = {'resampler__sampling_strategy' : ratios}
    grid = GridSearchCV(pipeline, params,cv=5,scoring='roc_auc')
    grid.fit(X_train, y_train)
    pRatio = grid.best_params_['resampler__sampling_strategy']
    ROS_instance = RandomOverSampler(sampling_strategy=pRatio)
    X_resampled, y_resampled = ROS_instance.fit_sample(X_train, y_train)
    stat = reduce_classify(X_resampled,y_resampled,X_test,y_test,'ROS', grid.best_params_,encoder,classifier)
    results = results.append(( stat ))

    
    resampler_instance = SMOTE()
    pipeline = Pipeline([('resampler', resampler_instance),  ('clf', classifier)])
    params = {'resampler__sampling_strategy' : ratios,
             'resampler__k_neighbors': k_neighbors}
    grid = GridSearchCV(pipeline, params,cv=5,scoring='roc_auc')
    grid.fit(X_train, y_train)
    pSS = grid.best_params_['resampler__sampling_strategy']
    pKN = grid.best_params_['resampler__k_neighbors']
    resampler_instance = SMOTE(sampling_strategy=pSS, k_neighbors = pKN)
    X_resampled, y_resampled = resampler_instance.fit_sample(X_train, y_train)
    stat = reduce_classify(X_resampled,y_resampled,X_test,y_test,'SMT', grid.best_params_,encoder,classifier)
    results = results.append(( stat ))
    
    
    resampler_instance = ADASYN()
    pipeline = Pipeline([('resampler', resampler_instance), ('clf', classifier)])
    params = {'resampler__sampling_strategy' : ratios,
             'resampler__n_neighbors': k_neighbors}
    grid = GridSearchCV(pipeline, params,cv=5,scoring='roc_auc')
    grid.fit(X_train, y_train)
    pRatio = grid.best_params_['resampler__sampling_strategy']
    pKN = grid.best_params_['resampler__n_neighbors']
    resampler_instance = ADASYN(sampling_strategy=pRatio,n_neighbors=pKN)
    X_resampled, y_resampled = resampler_instance.fit_sample(X_train, y_train)
    stat = reduce_classify(X_resampled,y_resampled,X_test,y_test,'ADA', grid.best_params_,encoder,classifier)
    results = results.append(( stat ))
    
    
    resampler_instance = BorderlineSMOTE()
    pipeline = Pipeline([('resampler', resampler_instance),  ('clf', classifier)])
    params = {'resampler__sampling_strategy' : ratios,
             'resampler__k_neighbors': k_neighbors,
             'resampler__kind':['borderline-1','borderline-2']}
    grid = GridSearchCV(pipeline, params,cv=5,scoring='roc_auc')
    grid.fit(X_train, y_train)
    pSS = grid.best_params_['resampler__sampling_strategy']
    pKN = grid.best_params_['resampler__k_neighbors']
    pKnd = grid.best_params_['resampler__kind']
    resampler_instance = BorderlineSMOTE(sampling_strategy=pSS,k_neighbors=pKN,kind=pKnd)
    X_resampled, y_resampled = resampler_instance.fit_sample(X_train, y_train)
    stat = reduce_classify(X_resampled,y_resampled,X_test,y_test,'SMT_BL',grid.best_params_,encoder,classifier)
    results = results.append(( stat ))
    
    resampler_instance = SMOTEENN()
    pipeline = Pipeline([('resampler', resampler_instance), ('clf', classifier)])
    params = {'resampler__sampling_strategy' : ratios}
    grid = GridSearchCV(pipeline, params,cv=5,scoring='roc_auc')
    grid.fit(X_train, y_train)
    pRatio = grid.best_params_['resampler__sampling_strategy']
    resampler_instance = SMOTEENN(sampling_strategy=pRatio)
    X_resampled, y_resampled = resampler_instance.fit_sample(X_train, y_train)
    stat = reduce_classify(X_resampled,y_resampled,X_test,y_test,'SMT_ENN', grid.best_params_,encoder,classifier)
    results = results.append(( stat ))
    
    
    resampler_instance = SMOTETomek()
    pipeline = Pipeline([('resampler', resampler_instance),  ('clf', classifier)])
    params = {'resampler__sampling_strategy' : ratios}
    grid = GridSearchCV(pipeline, params,cv=5,scoring='roc_auc')
    grid.fit(X_train, y_train)
    pRatio = grid.best_params_['resampler__sampling_strategy']
    resampler_instance = SMOTETomek(sampling_strategy=pRatio)
    X_resampled, y_resampled = resampler_instance.fit_sample(X_train, y_train)
    stat = reduce_classify(X_resampled,y_resampled,X_test,y_test,'SMT_TL', grid.best_params_,encoder,classifier)
    results = results.append(( stat ))
    
    
    resampler_instance = RandomUnderSampler()
    pipeline = Pipeline([('resampler', resampler_instance),  ('clf', classifier)])
    params = {'resampler__sampling_strategy' : ratios}
    grid = GridSearchCV(pipeline, params,cv=5,scoring='roc_auc')
    grid.fit(X_train, y_train)
    pRatio = grid.best_params_['resampler__sampling_strategy']
    resampler_instance = RandomUnderSampler(sampling_strategy=pRatio)
    X_resampled, y_resampled = resampler_instance.fit_sample(X_train, y_train)
    stat = reduce_classify(X_resampled,y_resampled,X_test,y_test,'RUS', grid.best_params_,encoder,classifier)
    results = results.append(( stat ))
    

    resampler_instance = NeighbourhoodCleaningRule()
    pipeline = Pipeline([('resampler', resampler_instance), ('clf', classifier)])
    params = {'resampler__sampling_strategy' : ['not minority','not majority','all'],
              'resampler__n_neighbors':[3,6,9]}
    grid = GridSearchCV(pipeline, params,cv=5,scoring='roc_auc')
    grid.fit(X_train, y_train)
    pRatio = grid.best_params_['resampler__sampling_strategy']
    pN = grid.best_params_['resampler__n_neighbors']
    resampler_instance = NeighbourhoodCleaningRule(sampling_strategy=pRatio,n_neighbors=pN)
    X_resampled, y_resampled = resampler_instance.fit_sample(X_train, y_train)
    stat = reduce_classify(X_resampled,y_resampled,X_test,y_test,'NCL',grid.best_params_,encoder,classifier)
    results = results.append(( stat ))

    
    resampler_instance = TomekLinks()
    pipeline = Pipeline([('resampler', resampler_instance),  ('clf', classifier)])
    params = {'resampler__sampling_strategy' : ['not minority','not majority','all']}
    grid = GridSearchCV(pipeline, params,cv=5,scoring='roc_auc')
    grid.fit(X_train, y_train)
    pRatio = grid.best_params_['resampler__sampling_strategy']
    resampler_instance = TomekLinks(sampling_strategy=pRatio)
    X_resampled, y_resampled = resampler_instance.fit_sample(X_train, y_train)
    stat = reduce_classify(X_resampled,y_resampled,X_test,y_test,'TL', grid.best_params_,encoder,classifier)
    results = results.append(( stat ))

    return results
    

In [ ]:
def resample_classify(X_train,X_test, y_train,y_test, ratios,encoder,classifier):
    results = pd.DataFrame()
    k_neighbors = [5,10,15]

    
    resampler_instance = OneSidedSelection()
    pipeline = Pipeline([('resampler', resampler_instance),  ('clf', classifier)])
    params = {'resampler__sampling_strategy' : ['not minority','not majority','all'],
              'resampler__n_neighbors':[1,3,6],
             'resampler__n_seeds_S':[10,100,200]}
    grid = GridSearchCV(pipeline, params,cv=5,scoring='roc_auc')
    grid.fit(X_train, y_train)
    pRatio = grid.best_params_['resampler__sampling_strategy']
    pNeighbors = grid.best_params_['resampler__n_neighbors']
    pSeeds = grid.best_params_['resampler__n_seeds_S']
    resampler_instance = OneSidedSelection(sampling_strategy=pRatio,n_neighbors=pNeighbors,n_seeds_S=pSeeds)
    X_resampled, y_resampled = resampler_instance.fit_sample(X_train, y_train)
    stat = classify(X_resampled,y_resampled,X_test,y_test,'OSS',grid.best_params_ ,'NA','NA',encoder,classifier)
    results = results.append(( stat ))

    resampler_instance = RandomOverSampler()
    pipeline = Pipeline([('resampler', resampler_instance), ('clf', classifier)])
    params = {'resampler__sampling_strategy' : ratios}
    grid = GridSearchCV(pipeline, params,cv=5,scoring='roc_auc')
    grid.fit(X_train, y_train)
    pRatio = grid.best_params_['resampler__sampling_strategy']
    resampler_instance = RandomOverSampler(sampling_strategy=pRatio)
    X_resampled, y_resampled = resampler_instance.fit_sample(X_train, y_train)
    stat = classify(X_resampled,y_resampled,X_test,y_test,'ROS', grid.best_params_,'NA','NA',encoder,classifier)
    results = results.append(( stat ))

    resampler_instance = SMOTE()
    pipeline = Pipeline([('resampler', resampler_instance),  ('clf', classifier)])
    params = {'resampler__sampling_strategy' : ratios,
             'resampler__k_neighbors': k_neighbors}
    grid = GridSearchCV(pipeline, params,cv=5,scoring='roc_auc')
    grid.fit(X_train, y_train)
    pSS = grid.best_params_['resampler__sampling_strategy']
    pKN = grid.best_params_['resampler__k_neighbors']
    resampler_instance = SMOTE(sampling_strategy=pSS, k_neighbors = pKN)
    X_resampled, y_resampled = resampler_instance.fit_sample(X_train, y_train)
    stat = classify(X_resampled,y_resampled,X_test,y_test,'SMT', grid.best_params_,'NA','NA',encoder,classifier)
    results = results.append(( stat ))
    
    
    resampler_instance = ADASYN()
    pipeline = Pipeline([('resampler', resampler_instance), ('clf', classifier)])
    params = {'resampler__sampling_strategy' : ratios,
             'resampler__n_neighbors': k_neighbors}
    grid = GridSearchCV(pipeline, params,cv=5,scoring='roc_auc')
    grid.fit(X_train, y_train)
    pRatio = grid.best_params_['resampler__sampling_strategy']
    pKN = grid.best_params_['resampler__n_neighbors']
    resampler_instance = ADASYN(sampling_strategy=pRatio,n_neighbors=pKN)
    X_resampled, y_resampled = resampler_instance.fit_sample(X_train, y_train)
    stat = classify(X_resampled,y_resampled,X_test,y_test,'ADA', grid.best_params_,'NA','NA',encoder,classifier)
    results = results.append(( stat ))
    
    resampler_instance = BorderlineSMOTE()
    pipeline = Pipeline([('resampler', resampler_instance),  ('clf', classifier)])
    params = {'resampler__sampling_strategy' : ratios,
             'resampler__k_neighbors': k_neighbors,
             'resampler__kind':['borderline-1','borderline-2']}
    grid = GridSearchCV(pipeline, params,cv=5,scoring='roc_auc')
    grid.fit(X_train, y_train)
    pSS = grid.best_params_['resampler__sampling_strategy']
    pKN = grid.best_params_['resampler__k_neighbors']
    pKnd = grid.best_params_['resampler__kind']
    resampler_instance = BorderlineSMOTE(sampling_strategy=pSS,k_neighbors=pKN,kind=pKnd)
    X_resampled, y_resampled = resampler_instance.fit_sample(X_train, y_train)
    stat = classify(X_resampled,y_resampled,X_test,y_test,'SMT_BL',grid.best_params_,'NA','NA',encoder,classifier)
    results = results.append(( stat ))
    
    resampler_instance = SMOTEENN()
    pipeline = Pipeline([('resampler', resampler_instance), ('clf', classifier)])
    params = {'resampler__sampling_strategy' : ratios}
    grid = GridSearchCV(pipeline, params,cv=5,scoring='roc_auc')
    grid.fit(X_train, y_train)
    pRatio = grid.best_params_['resampler__sampling_strategy']
    resampler_instance = SMOTEENN(sampling_strategy=pRatio)
    X_resampled, y_resampled = resampler_instance.fit_sample(X_train, y_train)
    stat = classify(X_resampled,y_resampled,X_test,y_test,'SMT_ENN', grid.best_params_,'NA','NA',encoder,classifier)
    results = results.append(( stat ))
    
    resampler_instance = SMOTETomek()
    pipeline = Pipeline([('resampler', resampler_instance),  ('clf', classifier)])
    params = {'resampler__sampling_strategy' : ratios}
    grid = GridSearchCV(pipeline, params,cv=5,scoring='roc_auc')
    grid.fit(X_train, y_train)
    pRatio = grid.best_params_['resampler__sampling_strategy']
    resampler_instance = SMOTETomek(sampling_strategy=pRatio)
    X_resampled, y_resampled = resampler_instance.fit_sample(X_train, y_train)
    stat = classify(X_resampled,y_resampled,X_test,y_test,'SMT_TL', grid.best_params_,'NA','NA',encoder,classifier)
    results = results.append(( stat ))
    
    resampler_instance = RandomUnderSampler()
    pipeline = Pipeline([('resampler', resampler_instance),  ('clf', classifier)])
    params = {'resampler__sampling_strategy' : ratios}
    grid = GridSearchCV(pipeline, params,cv=5,scoring='roc_auc')
    grid.fit(X_train, y_train)
    pRatio = grid.best_params_['resampler__sampling_strategy']
    resampler_instance = RandomUnderSampler(sampling_strategy=pRatio)
    X_resampled, y_resampled = resampler_instance.fit_sample(X_train, y_train)
    stat = classify(X_resampled,y_resampled,X_test,y_test,'RUS', grid.best_params_,'NA','NA',encoder,classifier)
    results = results.append(( stat ))
    
    
    resampler_instance = NeighbourhoodCleaningRule()
    pipeline = Pipeline([('resampler', resampler_instance), ('clf', classifier)])
    params = {'resampler__sampling_strategy' : ['not minority','not majority','all'],
              'resampler__n_neighbors':[3,6,9]}
    grid = GridSearchCV(pipeline, params,cv=5,scoring='roc_auc')
    grid.fit(X_train, y_train)
    pRatio = grid.best_params_['resampler__sampling_strategy']
    pN = grid.best_params_['resampler__n_neighbors']
    resampler_instance = NeighbourhoodCleaningRule(sampling_strategy=pRatio,n_neighbors=pN)
    X_resampled, y_resampled = resampler_instance.fit_sample(X_train, y_train)
    stat = classify(X_resampled,y_resampled,X_test,y_test,'NCL',grid.best_params_,'NA','NA',encoder,classifier)
    results = results.append(( stat ))

    
    resampler_instance = TomekLinks()
    pipeline = Pipeline([('resampler', resampler_instance),  ('clf', classifier)])
    params = {'resampler__sampling_strategy' : ['not minority','not majority','all']}
    grid = GridSearchCV(pipeline, params,cv=5,scoring='roc_auc')
    grid.fit(X_train, y_train)
    pRatio = grid.best_params_['resampler__sampling_strategy']
    resampler_instance = TomekLinks(sampling_strategy=pRatio)
    X_resampled, y_resampled = resampler_instance.fit_sample(X_train, y_train)
    stat = classify(X_resampled,y_resampled,X_test,y_test,'TL', grid.best_params_,'NA','NA',encoder,classifier)
    results = results.append(( stat ))

    return results

In [ ]:
def reduce_classify(X_train,y_train,X_test,y_test,resampler_name, parameterR,encoder,classifier):

    #take different percentages of columns with 3 steps
    n=[math.ceil(len(X_train.columns)*0.7),
    math.ceil(len(X_train.columns)*0.5),
    math.ceil(len(X_train.columns)*0.3)]

    ft_stat = pd.DataFrame()

    transformer = 'PCA'
    pca_instance = PCA()
    pipeline = Pipeline([('transformer', pca_instance), ('clf', classifier)])
    params = {'transformer__n_components' : n}
    grid = GridSearchCV(pipeline, params,cv=5,scoring='roc_auc')
    best_model = grid.fit(X_train, y_train)
    parameter_n_comp = grid.best_params_['transformer__n_components']
    pca = PCA(n_components=parameter_n_comp)
    X_train_principalComponents = pca.fit_transform(X_train)
    X_test_principalComponents = pca.fit_transform(X_test)
    stats = classify(X_train_principalComponents,y_train,X_test_principalComponents,y_test,resampler_name,parameterR, transformer, grid.best_params_,encoder,classifier)
    ft_stat = ft_stat.append(stats)

    transformer = 'CFS'
    selectKBest_instance = SelectKBest()
    pipeline = Pipeline([('transformer', selectKBest_instance),  ('clf', classifier)])
    params = {'transformer__k' : n}
    grid = GridSearchCV(pipeline, params,cv=5,scoring='roc_auc')
    best_model = grid.fit(X_train, y_train)
    parameterT = grid.best_params_['transformer__k']
    KBest = SelectKBest( k=parameterT)

    kbest_selector = KBest.fit(X_train, y_train)
    
    X_KBestFeatures_train = kbest_selector.transform(X_train)
    X_KBestFeatures_test = kbest_selector.transform(X_test)
    stats = classify(X_KBestFeatures_train,y_train,X_KBestFeatures_test,y_test,resampler_name,parameterR, transformer, grid.best_params_,encoder,classifier)
    ft_stat = ft_stat.append(stats)


    return ft_stat


In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calculate_vif(X, thresh):
    # Taken from https://stats.stackexchange.com/a/253620/53565 and modified
    dropped=True
    while dropped:
        variables = X.columns
        dropped = False
        vif = [variance_inflation_factor(X[variables].values, X.columns.get_loc(var)) for var in X.columns]
        max_vif = max(vif)
        if max_vif > thresh:
            maxloc = vif.index(max_vif)
            print(f'Dropping {X.columns[maxloc]} with vif={max_vif}')
            X = X.drop([X.columns.tolist()[maxloc]], axis=1)
            dropped=True
    return X

def removeMulticolinearFeatures(X_train, X_test, vif):
    X_train = calculate_vif(X_train, vif)
    columns_list = list(X_train)
    X_test = X_test[columns_list]
    return X_train, X_test

In [ ]:
#fmin(objective function, parameters list, optimizing algorithm)
#objective function returns a value that has to be minimazied
#tpe.suggest runs the Tree-structured Parzen estimator
best = 0
max_evals = 100
def acc_model(params,classifier):
    clf = classifier(**params)
    return cross_val_score(clf, X_train, y_train,scoring = 'roc_auc').mean()

def f_rf(params):
    global best
    classifier = RandomForestClassifier
    acc = acc_model(params,classifier)
    if acc > best:
        best = acc
    return {'loss': -acc, 'status': STATUS_OK}

def getRF():
    criterion_list = ["gini", "entropy"]
    max_features_list = ['auto','sqrt','log2','None']
    param_space = {
        'max_depth': hp.choice('max_depth', range(1,30)),
        'max_features': hp.choice('max_features', ['sqrt','log2','None']),
        'n_estimators': hp.choice('n_estimators', range(50,150)),
        'criterion': hp.choice('criterion', ["gini", "entropy"])
    }
        
    trials = Trials()
    best = fmin(f_rf, param_space, algo=tpe.suggest, max_evals=max_evals, trials=trials)
    print ('best:')
    print (best)  
    criterion = criterion_list[best['criterion']]
    if best['max_depth'] == 0:
        max_depth = None
    else: 
        max_depth = best['max_depth']
    max_features = max_features_list[best['max_features']]
    n_estimators = best['n_estimators']
    optimizedRF = RandomForestClassifier(criterion = criterion, max_depth=max_depth,max_features = max_features,n_estimators=n_estimators)
    return optimizedRF,best

def f_lr(params):
    global best
    classifier = LogisticRegression
    acc = acc_model(params,classifier)
    if acc > best:
        best = acc
    return {'loss': -acc, 'status': STATUS_OK}

def getLR():
    penalty_list = ['l1','l2','elasticnet','none']
    param_space = {
        'C' : hp.uniform('C', 0.001, 100),
        'penalty' : hp.choice('penalty', ['l1','l2','elasticnet','none']) 
    }
        
    trials = Trials()
    best = fmin(f_lr, param_space, algo=tpe.suggest, max_evals=max_evals, trials=trials)
    print ('best:')
    print (best)  
    penalty = penalty_list[best['penalty']]
    C = best['C']
    optimizedLR = LogisticRegression(C=C,penalty=penalty)
    return optimizedLR,best

def f_mlp(params):
    global best
    classifier = MLPClassifier
    acc = acc_model(params,classifier)
    if acc > best:
        best = acc
    return {'loss': -acc, 'status': STATUS_OK}

def getMLP():
    param_space = {
        'alpha': hp.uniform('alpha',0.0001,1),
        'hidden_layer_sizes': hp.randint('hidden_layer_sizes',100)
    }
    trials = Trials()
    best = fmin(f_mlp, param_space, algo=tpe.suggest, max_evals=max_evals, trials=trials)
    print ('best:')
    print (best)
    alpha = best['alpha']
    hidden_layer_sizes = best['hidden_layer_sizes']
    optimizedMLP = MLPClassifier(hidden_layer_sizes=(hidden_layer_sizes),
                                 alpha=alpha)
    return optimizedMLP,best


In [ ]:
preprocess_technique_stat = pd.DataFrame()
dataset_name = 'LD'
ld = pd.read_csv (r'C:\Users\ihorh\Desktop\Master Thesis\data_ihorirene\Thomas (2002)\Loan Data.csv',delimiter=';')

#drop duplicates
ld = ld.drop_duplicates(keep='first')

# #split dataset into data and target
y = ld.BAD
X = ld.drop('BAD',axis=1)


#data set specific ratios to resample
ratios =[0.406,0.452,0.5]

toScaleColumns = ['YOB','DAINC','DHVAL','DMORT','DOUTM','DOUTL','DOUTHP','DOUTCC','SINC']

X_train_original, X_test_original, y_train_original, y_test_original = split(X,y)

#treat missing values
imp_cat = SimpleImputer(strategy="most_frequent")
imp_num = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train_num = X_train_original.select_dtypes(include=['int64', 'float64'])
X_test_num = X_test_original.select_dtypes(include=['int64', 'float64'])
X_train_cat = X_train_original.select_dtypes(exclude=['int64', 'float64'])
X_test_cat = X_test_original.select_dtypes(exclude=['int64', 'float64'])

#impute mean mode
X_train_cat = pd.DataFrame(imp_cat.fit_transform(X_train_cat),columns=X_train_cat.columns)
X_test_cat = pd.DataFrame(imp_cat.transform(X_test_cat),columns=X_test_cat.columns)
X_train_num = pd.DataFrame(imp_num.fit_transform(X_train_num),columns = X_train_num.columns)
X_test_num = pd.DataFrame(imp_num.transform(X_test_num),columns=X_test_num.columns)

X_train_original = pd.concat([X_train_cat,X_train_num],axis=1)
X_test_original = pd.concat([X_test_cat,X_test_num],axis=1)




#scale
X_train_original[toScaleColumns] = scaler.fit_transform(X_train_original[toScaleColumns] )
X_test_original[toScaleColumns]  = scaler.transform(X_test_original[toScaleColumns] )

#downcast numerical columns
for column in toScaleColumns:
    X_train_original[column] = pd.to_numeric(X_train_original[column], downcast='float')




###ONE HOT ENCODING-------------------------------------------------
X_train = one_hot_encode(X_train_original)
X_test = one_hot_encode(X_test_original)
y_train = y_train_original
y_test = y_test_original
columns_list = X_train.columns.tolist()
X_test = X_test[X_test.columns.intersection(columns_list)]


#add columns that are not present in test
for column in columns_list:
    if column not in X.columns:
        X_test[column] = 0


#reduce colinearity
X_train, X_test = removeMulticolinearFeatures(X_train,X_test, 10)


#classifier optimization is performed on OHE dataset
RF, bestParametersRF = getRF()
LR, bestParametersLR= getLR()
MLP, bestParametersMLP = getMLP()
classifiers = [LR,RF,MLP]
bestParametersList=[str(bestParametersLR),str(bestParametersRF),str(bestParametersMLP)]


i = 0
for classifier in classifiers:
    stat = classify_master(X_train, X_test, y_train, y_test,'OHE',classifier)
    stat['ClassifierParameters'] = bestParametersList[i]
    preprocess_technique_stat = preprocess_technique_stat.append(stat)
    i=i+1


###END ONE HOT ENCODING-------------------------------------------------

###WOE ENCODING-----------------------------------------------
X_train = X_train_original
X_test = X_test_original
y_train = y_train_original
y_test = y_test_original

#Encode WOE
columns_list = X_train.columns.tolist()
for column in columns_list:
    if is_string_dtype(X_train[column]):
        encoder = ce.WOEEncoder(cols=[column])
        encoder.fit(X_train,y_train)
        X_train = encoder.transform(X_train)
        X_test = encoder.transform(X_test)
        
#reduce colinearity
X_train, X_test = removeMulticolinearFeatures(X_train,X_test, 10)



# #classifier optimization is performed on WOE dataset
RF, bestParametersRF = getRF()
LR, bestParametersLR= getLR()
MLP, bestParametersMLP = getMLP()
classifiers = [LR,RF,MLP]
bestParametersList=[str(bestParametersLR),str(bestParametersRF),str(bestParametersMLP)]


i = 0
for classifier in classifiers:
    stat = classify_master(X_train, X_test, y_train, y_test,'WOE',classifier)
    stat['ClassifierParameters'] = bestParametersList[i]
    preprocess_technique_stat = preprocess_technique_stat.append(stat)
    i=i+1

###EMB---------------------------------
X_train = X_train_original
y_train = y_train_original
X_test = X_test_original
y_test = y_test_original


        
embedding_info = cem.get_embedding_info(X_train)

X_encoded,encoders = cem.get_label_encoded_data(X_train)



embeddings = cem.get_embeddings(X_encoded, y_train, categorical_embedding_info=embedding_info, 
                            is_classification=True, epochs=100,batch_size=256)




X_train = cem.fit_transform(X_train, embeddings=embeddings, encoders=encoders, drop_categorical_vars=True)
X_test = cem.fit_transform(X_test, embeddings=embeddings, encoders=encoders, drop_categorical_vars=True)
 

#reduce colinearity
X_train, X_test = removeMulticolinearFeatures(X_train,X_test, 10)

# #classifier optimization is performed on OHE dataset
RF, bestParametersRF = getRF()
LR, bestParametersLR= getLR()
MLP, bestParametersMLP = getMLP()
classifiers = [LR,RF,MLP]
bestParametersList=[str(bestParametersLR),str(bestParametersRF),str(bestParametersMLP)]

i = 0
for classifier in classifiers:
    stat = classify_master(X_train, X_test, y_train, y_test,'EMB',classifier)
    stat['ClassifierParameters'] = bestParametersList[i]
    preprocess_technique_stat = preprocess_technique_stat.append(stat)
    i=i+1
    
###END EMB-------------------------------------------------------------
preprocess_technique_stat.to_excel(r'C:\Users\ihorh\Desktop\Master Thesis\stats\prep_technique_stats_'+dataset_name+'.xlsx')  
   
l_stat = preprocess_technique_stat




In [ ]:
preprocess_technique_stat = pd.DataFrame()
dataset_name = 'HMEQ'
hmeq = pd.read_csv (r'C:\Users\ihorh\Desktop\Master Thesis\data_ihorirene\hmeq\hmeq.csv'
                 ,delimiter=';')

#drop duplicates
hmeq = hmeq.drop_duplicates(keep='first')


#convert to month scale
hmeq[['CLAGE']] = round(hmeq[['CLAGE']]*12,1)


#split dataset into data and target
y = hmeq.BAD
X = hmeq.drop('BAD',axis=1)


#data set specific ratios to resample
ratios =[0.333,0.416,0.5]

toScaleColumns = ['LOAN','MORTDUE','VALUE','CLAGE','DEBTINC']

X_train_original, X_test_original, y_train_original, y_test_original = split(X,y)

#treat missing values
imp_cat = SimpleImputer(strategy="most_frequent")
imp_num = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train_num = X_train_original.select_dtypes(include=['int64', 'float64'])
X_test_num = X_test_original.select_dtypes(include=['int64', 'float64'])
X_train_cat = X_train_original.select_dtypes(exclude=['int64', 'float64'])
X_test_cat = X_test_original.select_dtypes(exclude=['int64', 'float64'])

#impute mean mode
X_train_cat = pd.DataFrame(imp_cat.fit_transform(X_train_cat),columns=X_train_cat.columns)
X_test_cat = pd.DataFrame(imp_cat.transform(X_test_cat),columns=X_test_cat.columns)
X_train_num = pd.DataFrame(imp_num.fit_transform(X_train_num),columns = X_train_num.columns)
X_test_num = pd.DataFrame(imp_num.transform(X_test_num),columns=X_test_num.columns)

X_train_original = pd.concat([X_train_cat,X_train_num],axis=1)
X_test_original = pd.concat([X_test_cat,X_test_num],axis=1)




#scale
X_train_original[toScaleColumns] = scaler.fit_transform(X_train_original[toScaleColumns] )
X_test_original[toScaleColumns]  = scaler.transform(X_test_original[toScaleColumns] )

#downcast numerical columns
for column in toScaleColumns:
    X_train_original[column] = pd.to_numeric(X_train_original[column], downcast='float')


###ONE HOT ENCODING-------------------------------------------------
X_train = one_hot_encode(X_train_original)
X_test = one_hot_encode(X_test_original)
y_train = y_train_original
y_test = y_test_original
columns_list = X_train.columns.tolist()
X_test = X_test[X_test.columns.intersection(columns_list)]


#add columns that are not present in test
for column in columns_list:
    if column not in X.columns:
        X_test[column] = 0


#reduce colinearity
X_train, X_test = removeMulticolinearFeatures(X_train,X_test, 10)


#classifier optimization is performed on OHE dataset
RF, bestParametersRF = getRF()
LR, bestParametersLR= getLR()
MLP, bestParametersMLP = getMLP()
classifiers = [LR,RF,MLP]
bestParametersList=[str(bestParametersLR),str(bestParametersRF),str(bestParametersMLP)]


i = 0
for classifier in classifiers:
    stat = classify_master(X_train, X_test, y_train, y_test,'OHE',classifier)
    stat['ClassifierParameters'] = bestParametersList[i]
    preprocess_technique_stat = preprocess_technique_stat.append(stat)
    i=i+1


###END ONE HOT ENCODING-------------------------------------------------


###WOE ENCODING-----------------------------------------------
X_train = X_train_original
X_test = X_test_original
y_train = y_train_original
y_test = y_test_original

#Encode WOE
columns_list = X_train.columns.tolist()
for column in columns_list:
    if is_string_dtype(X_train[column]):
        encoder = ce.WOEEncoder(cols=[column])
        encoder.fit(X_train,y_train)
        X_train = encoder.transform(X_train)
        X_test = encoder.transform(X_test)
        
#reduce colinearity
X_train, X_test = removeMulticolinearFeatures(X_train,X_test, 10)



# #classifier optimization is performed on WOE dataset
RF, bestParametersRF = getRF()
LR, bestParametersLR= getLR()
MLP, bestParametersMLP = getMLP()
classifiers = [LR,RF,MLP]
bestParametersList=[str(bestParametersLR),str(bestParametersRF),str(bestParametersMLP)]


i = 0
for classifier in classifiers:
    stat = classify_master(X_train, X_test, y_train, y_test,'WOE',classifier)
    stat['ClassifierParameters'] = bestParametersList[i]
    preprocess_technique_stat = preprocess_technique_stat.append(stat)
    i=i+1

###EMB---------------------------------
X_train = X_train_original
y_train = y_train_original
X_test = X_test_original
y_test = y_test_original


        
embedding_info = cem.get_embedding_info(X_train)

X_encoded,encoders = cem.get_label_encoded_data(X_train)



embeddings = cem.get_embeddings(X_encoded, y_train, categorical_embedding_info=embedding_info, 
                            is_classification=True, epochs=100,batch_size=256)




X_train = cem.fit_transform(X_train, embeddings=embeddings, encoders=encoders, drop_categorical_vars=True)
X_test = cem.fit_transform(X_test, embeddings=embeddings, encoders=encoders, drop_categorical_vars=True)
 

#reduce colinearity
X_train, X_test = removeMulticolinearFeatures(X_train,X_test, 10)

# #classifier optimization is performed on OHE dataset
RF, bestParametersRF = getRF()
LR, bestParametersLR= getLR()
MLP, bestParametersMLP = getMLP()
classifiers = [LR,RF,MLP]
bestParametersList=[str(bestParametersLR),str(bestParametersRF),str(bestParametersMLP)]

i = 0
for classifier in classifiers:
    stat = classify_master(X_train, X_test, y_train, y_test,'EMB',classifier)
    stat['ClassifierParameters'] = bestParametersList[i]
    preprocess_technique_stat = preprocess_technique_stat.append(stat)
    i=i+1
    
###END EMB-------------------------------------------------------------
preprocess_technique_stat.to_excel(r'C:\Users\ihorh\Desktop\Master Thesis\stats\prep_technique_stats_'+dataset_name+'.xlsx')  
   
h_stat = preprocess_technique_stat



In [ ]:
preprocess_technique_stat = pd.DataFrame()
dataset_name = 'bene'

#data set specific ratios to resample
ratios =[0.166,0.332,0.5]
bene = pd.read_csv (r'C:\Users\ihorh\Desktop\Master Thesis\data_ihorirene\bene3\data_bene3.csv'
                 ,delimiter=';')
#drop duplicates
bene = bene.drop_duplicates(keep='first')


bene = bene.drop(columns=['account_id'])
bene = bene.rename(columns={'default': 'BAD'})


#treat values based on their meaning
bene.loc[bene.HWGV_REL_MD_AT > 900, 'HWGV_REL_MD_AT'] = np.nan

bene.replace({'HGESL_KD' : { 2: 'm',1: 'f'}},inplace=True)
bene.loc[bene.HWOON_STA_MD_AT == 99998, 'HWOON_STA_MD_AT'] = bene.HLTD_MD_AT

bene.replace({'HBURG_KD' : { 0: 'NA', 1 : 'Divorced', 2 : 'Married',3:'Not married',4:'Legally divorced',5:'Widow',6:'Overleden' }},inplace=True)

bene.replace({'HWOON_STA_KD' : {1 : 'Owner', 2 : 'Renter',3:'Lives at parents',4:'Other'}},inplace=True)
bene.replace({'HNAT_KD' : {1 : 'Belgium', 2 : 'West-Europe',3:'Rest of Europe',4:'North America/ Australia',5:'South/ Central America',6:'Asia/ Middle East',7:'Africa',8:'Not applicable'}},inplace=True)


bene[['HLTD_MD_AT']] = round(bene[['HLTD_MD_AT']]/12,1)
bene[['HWOON_STA_MD_AT']] = round(bene[['HWOON_STA_MD_AT']]/12,1)
bene[['HWGV_REL_MD_AT']] = round(bene[['HWGV_REL_MD_AT']]/12,1)
  
bene.rename(columns={'HGESL_KD': 'Gender','HLTD_MD_AT': 'Age','HWOON_STA_MD_AT':'LivesWithParents',
                                 'HWGV_REL_MD_AT':'WorksAtEmployer','HNAT_KD':'Origin','HBURG_KD':'CivilState'
                                ,'HWOON_STA_KD':'HomeOwnership','MOB':'Month'}, inplace=True)

#end treat values based on their meaning

# #split dataset into data and target
y = bene.BAD
X = bene.drop('BAD',axis=1)


toScaleColumns = ['amount_of_loan','Month','LivesWithParents','Age','WorksAtEmployer','DPD']


X_train_original, X_test_original, y_train_original, y_test_original = split(X,y)

#treat missing values
imp_cat = SimpleImputer(strategy="most_frequent")
imp_num = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train_num = X_train_original.select_dtypes(include=['int64', 'float64'])
X_test_num = X_test_original.select_dtypes(include=['int64', 'float64'])
X_train_cat = X_train_original.select_dtypes(exclude=['int64', 'float64'])
X_test_cat = X_test_original.select_dtypes(exclude=['int64', 'float64'])

#impute mean mode
X_train_cat = pd.DataFrame(imp_cat.fit_transform(X_train_cat),columns=X_train_cat.columns)
X_test_cat = pd.DataFrame(imp_cat.transform(X_test_cat),columns=X_test_cat.columns)
X_train_num = pd.DataFrame(imp_num.fit_transform(X_train_num),columns = X_train_num.columns)
X_test_num = pd.DataFrame(imp_num.transform(X_test_num),columns=X_test_num.columns)

X_train_original = pd.concat([X_train_cat,X_train_num],axis=1)
X_test_original = pd.concat([X_test_cat,X_test_num],axis=1)


#scale
X_train_original[toScaleColumns] = scaler.fit_transform(X_train_original[toScaleColumns] )
X_test_original[toScaleColumns]  = scaler.transform(X_test_original[toScaleColumns] )

#downcast numerical columns
for column in toScaleColumns:
    X_train_original[column] = pd.to_numeric(X_train_original[column], downcast='float')


###ONE HOT ENCODING-------------------------------------------------
X_train = one_hot_encode(X_train_original)
X_test = one_hot_encode(X_test_original)
y_train = y_train_original
y_test = y_test_original
columns_list = X_train.columns.tolist()
X_test = X_test[X_test.columns.intersection(columns_list)]




#add columns that are not present in test
for column in columns_list:
    if column not in X.columns:
        X_test[column] = 0


#reduce colinearity
X_train, X_test = removeMulticolinearFeatures(X_train,X_test, 10)




#classifier optimization is performed on OHE dataset
RF, bestParametersRF = getRF()
LR, bestParametersLR= getLR()
MLP, bestParametersMLP = getMLP()
classifiers = [LR,RF,MLP]
bestParametersList=[str(bestParametersLR),str(bestParametersRF),str(bestParametersMLP)]


i = 0
for classifier in classifiers:
    stat = classify_master(X_train, X_test, y_train, y_test,'OHE',classifier)
    stat['ClassifierParameters'] = bestParametersList[i]
    preprocess_technique_stat = preprocess_technique_stat.append(stat)
    i=i+1


###END ONE HOT ENCODING------------------------------------------


###WOE ENCODING-----------------------------------------------
X_train = X_train_original
X_test = X_test_original
y_train = y_train_original
y_test = y_test_original

#Encode WOE
columns_list = X_train.columns.tolist()
for column in columns_list:
    if is_string_dtype(X_train[column]):
        encoder = ce.WOEEncoder(cols=[column])
        encoder.fit(X_train,y_train)
        X_train = encoder.transform(X_train)
        X_test = encoder.transform(X_test)
        
#reduce colinearity
X_train, X_test = removeMulticolinearFeatures(X_train,X_test, 10)



# #classifier optimization is performed on WOE dataset
RF, bestParametersRF = getRF()
LR, bestParametersLR= getLR()
MLP, bestParametersMLP = getMLP()
classifiers = [LR,RF,MLP]
bestParametersList=[str(bestParametersLR),str(bestParametersRF),str(bestParametersMLP)]


i = 0
for classifier in classifiers:
    stat = classify_master(X_train, X_test, y_train, y_test,'WOE',classifier)
    stat['ClassifierParameters'] = bestParametersList[i]
    preprocess_technique_stat = preprocess_technique_stat.append(stat)
    i=i+1

###EMB---------------------------------
X_train = X_train_original
y_train = y_train_original
X_test = X_test_original
y_test = y_test_original


        
embedding_info = cem.get_embedding_info(X_train)

X_encoded,encoders = cem.get_label_encoded_data(X_train)



embeddings = cem.get_embeddings(X_encoded, y_train, categorical_embedding_info=embedding_info, 
                            is_classification=True, epochs=100,batch_size=256)




X_train = cem.fit_transform(X_train, embeddings=embeddings, encoders=encoders, drop_categorical_vars=True)
X_test = cem.fit_transform(X_test, embeddings=embeddings, encoders=encoders, drop_categorical_vars=True)
 

#reduce colinearity
X_train, X_test = removeMulticolinearFeatures(X_train,X_test, 10)

# #classifier optimization is performed on OHE dataset
RF, bestParametersRF = getRF()
LR, bestParametersLR= getLR()
MLP, bestParametersMLP = getMLP()
classifiers = [LR,RF,MLP]
bestParametersList=[str(bestParametersLR),str(bestParametersRF),str(bestParametersMLP)]

i = 0
for classifier in classifiers:
    stat = classify_master(X_train, X_test, y_train, y_test,'EMB',classifier)
    stat['ClassifierParameters'] = bestParametersList[i]
    preprocess_technique_stat = preprocess_technique_stat.append(stat)
    i=i+1
    
###END EMB-------------------------------------------------------------
preprocess_technique_stat.to_excel(r'C:\Users\ihorh\Desktop\Master Thesis\stats\prep_technique_stats_'+dataset_name+'.xlsx')  
    
b_stat = preprocess_technique_stat



In [ ]:
stats_joined = pd.concat([ l_stat,h_stat,b_stat],axis=1)
stats_joined.columns =['l_Dataset','l_Encoder','l_Resampler','l_Resampler_Parameters','l_Feature_Selection','l_FS_Parameters','l_Classifier','l_Recall','l_f1','l_AUC','l_BS','l_Classifier_Parameters',
                       'h_Dataset','h_Encoder','h_Resampler','h_Resampler_Parameters','h_Feature_Selection','h_FS_Parameters','h_Classifier','h_Recall','h_f1','h_AUC','h_BS','h_Classifier_Parameters',
                       'b_Dataset','b_Encoder','b_Resampler','b_Resampler_Parameters','b_Feature_Selection','b_FS_Parameters','b_Classifier','b_Recall','b_f1','b_AUC','b_BS','b_Classifier_Parameters']
stats_joined['l_Recall_r'] = stats_joined['l_Recall'].rank(method='dense',ascending=False)
stats_joined['l_f1_r'] = stats_joined['l_f1'].rank(method='dense',ascending=False)
stats_joined['l_AUC_r'] = stats_joined['l_AUC'].rank(method='dense',ascending=False)
stats_joined['l_BS_r'] = stats_joined['l_BS'].rank(method='dense')
stats_joined['h_Recall_r'] = stats_joined['h_Recall'].rank(method='dense',ascending=False)
stats_joined['h_f1_r'] = stats_joined['h_f1'].rank(method='dense',ascending=False)
stats_joined['h_AUC_r'] = stats_joined['h_AUC'].rank(method='dense',ascending=False)
stats_joined['h_BS_r'] = stats_joined['h_BS'].rank(method='dense')
stats_joined['b_Recall_r'] = stats_joined['b_Recall'].rank(method='dense',ascending=False)
stats_joined['b_f1_r'] = stats_joined['b_f1'].rank(method='dense',ascending=False)
stats_joined['b_AUC_r'] = stats_joined['b_AUC'].rank(method='dense',ascending=False)
stats_joined['b_BS_r'] = stats_joined['b_BS'].rank(method='dense')

stats_joined['AverageRank']= stats_joined[['l_Recall_r','l_f1_r','l_AUC_r','l_BS_r',
                                         'h_Recall_r','h_f1_r','h_AUC_r','h_BS_r'
                                         ,'b_Recall_r','b_f1_r','b_AUC_r','b_BS_r']].sum(axis=1)/12

stats_joined.to_excel(r'C:\Users\ihorh\Desktop\Master Thesis\stats\statistics_joined.xlsx')